# Código de tratamento dos dados ambientais do SEEG

In [26]:
## Load dependencies.
import os
import pandas as pd
import numpy as np
from global_function import strip_upper

In [21]:
## Read file.
df_original = pd.read_excel('bases-auxiliares/SEEG.xlsx', 0)

In [ ]:
df = df_original

# 1. Data Wrangling

Nesta etapa iremos limpar a base de dados e transformar os dados para um formato compatível com a base de dados de operações de financiamento.

# 1.1 - Exploração e limpeza do dataset

In [3]:
## Dataset overview.
df.head()

,NIVEL 1,NIVEL 2,NIVEL 3,NIVEL 4,NIVEL 5,NIVEL 6,TIPO DE EMISSÃO,GÁS,TERRITÓRIO,IBGE,...,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,AREA2019
0,Energia,NaN,NaN,NaN,NaN,NaN,Emissão,CO2e (t) GWP-AR5,AC,NaN,...,6.690951e+05,6.967727e+05,9.367922e+05,7.246570e+05,7.834469e+05,7.420719e+05,6.860833e+05,6.571363e+05,6.977519e+05,2827341.70
1,Agropecuária,NaN,NaN,NaN,NaN,NaN,Emissão,CO2e (t) GWP-AR5,AC,NaN,...,5.200435e+06,4.994819e+06,5.185200e+06,5.304081e+06,5.507291e+06,5.757112e+06,5.920841e+06,5.712448e+06,6.478450e+06,13950536.60
2,Processos Industriais,NaN,NaN,NaN,NaN,NaN,Emissão,CO2e (t) GWP-AR5,AC,NaN,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00
3,Resíduos,NaN,NaN,NaN,NaN,NaN,Emissão,CO2e (t) GWP-AR5,AC,NaN,...,2.415251e+05,2.586077e+05,2.907782e+05,3.126031e+05,3.285184e+05,3.549350e+05,3.699129e+05,3.847987e+05,4.012447e+05,2133611.48
4,Mudança de Uso da Terra e Floresta,NaN,NaN,NaN,NaN,NaN,Emissão,CO2e (t) GWP-AR5,AC,NaN,...,1.488266e+07,1.499894e+07,1.377548e+07,1.395242e+07,1.948930e+07,2.237926e+07,2.192239e+07,2.506700e+07,2.051789e+07,1985274.65


In [4]:
## Dataset dimensions
print(df.shape)

## NA' check
print(df.isna().sum())

## Veryfing duplicated records
print(df.duplicated().sum())


(280, 33)
NIVEL 1                  0
NIVEL 2                280
NIVEL 3                280
NIVEL 4                280
NIVEL 5                280
NIVEL 6                280
TIPO DE EMISSÃO          0
GÁS                      0
TERRITÓRIO              10
IBGE                   280
MUNICÍPIO              280
ATIVIDADE ECONÔMICA    280
PRODUTO                280
2000.0                   0
2001.0                   0
2002.0                   0
2003.0                   0
2004.0                   0
2005.0                   0
2006.0                   0
2007.0                   0
2008.0                   0
2009.0                   0
2010.0                   0
2011.0                   0
2012.0                   0
2013.0                   0
2014.0                   0
2015.0                   0
2016.0                   0
2017.0                   0
2018.0                   0
AREA2019                 0
dtype: int64
0


In [5]:
## Unique categories associated with CO2 emissions.
print(df['NIVEL 1'].unique())

## Unique types of emissions.
print(df['TIPO DE EMISSÃO'].unique())

## Unique types of gas emissions.
print(df['GÁS'].unique())



['Energia' 'Agropecuária' 'Processos Industriais' 'Resíduos '
 'Mudança de Uso da Terra e Floresta']
['Emissão' 'Remoção']
['CO2e (t) GWP-AR5']


In [6]:
## Selecting unused columns to drop.
drop_cols = ['NIVEL 2', 'NIVEL 3', 'NIVEL 4', 'NIVEL 5', 'NIVEL 6', 'GÁS', 'IBGE', 'MUNICÍPIO', 'ATIVIDADE ECONÔMICA', 'PRODUTO']

## Dropping from name index.
df.drop(drop_cols, axis = 1, inplace = True)

In [7]:
print('Shape antes de remover observações vazias do território:', df.shape)
## Dropping rows with NA in state identifier.
df.dropna(subset = ['TERRITÓRIO'], inplace = True)
print('Shape após remover observações vazias do território:', df.shape)
print(df.isna().sum())

Shape antes de remover observações vazias do território: (280, 23)
Shape após remover observações vazias do território: (270, 23)
NIVEL 1            0
TIPO DE EMISSÃO    0
TERRITÓRIO         0
2000.0             0
2001.0             0
2002.0             0
2003.0             0
2004.0             0
2005.0             0
2006.0             0
2007.0             0
2008.0             0
2009.0             0
2010.0             0
2011.0             0
2012.0             0
2013.0             0
2014.0             0
2015.0             0
2016.0             0
2017.0             0
2018.0             0
AREA2019           0
dtype: int64


In [8]:
df.head(20)

,NIVEL 1,TIPO DE EMISSÃO,TERRITÓRIO,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,...,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,AREA2019
0,Energia,Emissão,AC,8.160077e+05,8.938214e+05,1.072257e+06,1.064160e+06,8.259772e+05,7.625486e+05,6.468860e+05,...,6.690951e+05,6.967727e+05,9.367922e+05,7.246570e+05,7.834469e+05,7.420719e+05,6.860833e+05,6.571363e+05,6.977519e+05,2827341.70
1,Agropecuária,Emissão,AC,2.277725e+06,3.631115e+06,3.829350e+06,3.940962e+06,4.270671e+06,4.711030e+06,5.032734e+06,...,5.200435e+06,4.994819e+06,5.185200e+06,5.304081e+06,5.507291e+06,5.757112e+06,5.920841e+06,5.712448e+06,6.478450e+06,13950536.60
2,Processos Industriais,Emissão,AC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00
3,Resíduos,Emissão,AC,1.471381e+05,1.599258e+05,1.657446e+05,1.659423e+05,1.696912e+05,1.762751e+05,1.854000e+05,...,2.415251e+05,2.586077e+05,2.907782e+05,3.126031e+05,3.285184e+05,3.549350e+05,3.699129e+05,3.847987e+05,4.012447e+05,2133611.48
4,Mudança de Uso da Terra e Floresta,Emissão,AC,2.897559e+07,2.367799e+07,4.644432e+07,5.207100e+07,2.244174e+07,2.959129e+07,2.559454e+07,...,1.488266e+07,1.499894e+07,1.377548e+07,1.395242e+07,1.948930e+07,2.237926e+07,2.192239e+07,2.506700e+07,2.051789e+07,1985274.65
5,Energia,Remoção,AC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00
6,Agropecuária,Remoção,AC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00
7,Processos Industriais,Remoção,AC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00
8,Resíduos,Remoção,AC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00
9,Mudança de Uso da Terra e Floresta,Remoção,AC,-6.824883e+06,-7.124617e+06,-7.740340e+06,-8.894432e+06,-9.129781e+06,-1.053537e+07,-1.110335e+07,...,-1.182988e+07,-1.187147e+07,-1.195186e+07,-1.247986e+07,-1.268241e+07,-1.275190e+07,-1.279764e+07,-1.282119e+07,-1.291402e+07,1116735.35


In [9]:
## Unique states.
print('Total de estados: ', len(df['TERRITÓRIO'].unique()))

## Unique types of emission.
print('Total de tipos de emissão: ', len(df['TIPO DE EMISSÃO'].unique()))

## Unique nivel 1.
print('Total de categorias nivel 1: ', len(df['NIVEL 1'].unique()))


Total de estados:  27
Total de tipos de emissão:  2
Total de categorias nivel 1:  5


## 1.2 Transformação

Para mesclar os dados com o dataset de financiamento, precisamos deixar esses dados dispostos de uma maneira diferente.

Observando as informações da fase de exploração, percebemos que o dataset esta disposto da seguinte forma:
Para cada um dos 27 estados, temos 5 colunas com as categorias de nivel 1 com um tipo de emissão e 5 colunas com as categorias de nivel 1 para o outro tipo de emissão.

Desta forma, para cada estado, temos 10 observações que podem ou não ter os dados preenchidos para os anos entre 2000 e 2018.

Como os dados de financiamento estão dispostos com os anos em linhas, transformaremos este dataset para um formato com anos em linhas de acordo com as conclusões acima.



In [10]:
## Setting list of years.
years = np.arange(2000, 2020, 1)
print('Total de anos contemplados: ', len(years))

Total de anos contemplados:  20


In [11]:
## Slicing dataset: Return categorial information and numeric information in separate datasets.
df_cat = df.iloc[:, 0:3]
print('Dimensão do slice em categorias: ', df_cat.shape)

df_data = df.iloc[:,3:23]
print('Dimensão do slice em categorias: ', df_data.shape)

Dimensão do slice em categorias:  (270, 3)
Dimensão do slice em categorias:  (270, 20)


In [12]:
## Repeat categories equal to number of years.
df_cat = pd.DataFrame(np.repeat(df_cat.values, len(years), axis = 0))
print('Tamanho do novo dataset de categorias: ', len(df_cat))

Tamanho do novo dataset de categorias:  5400


Lembrando a forma como nossos dados estão dispostos (27 * 2 * 5) = 270 observações.
Multiplicamos esse dataset pelo número de anos na amostra, que tem um total de 270*20 = 5400 observações.

Dessa forma, iremos encaixar uma lista de anos (2000 a 2019) como uma nova coluna nesse dataset, onde serão armazenadas as informações númericas no novo formato.

In [13]:
## Converting years to a list
years = years.tolist()

## Creating the year list with the size of our categorical dataset.
years = years * (27 * 2 * 5)

print('Tamanho da lista de anos: ', len(years))


Tamanho da lista de anos:  5400


In [14]:
## Converting list to a new column in df_cat
df_cat['ANO'] = years

df_cat.head(40)

,0,1,2,ANO
0,Energia,Emissão,AC,2000
1,Energia,Emissão,AC,2001
2,Energia,Emissão,AC,2002
3,Energia,Emissão,AC,2003
4,Energia,Emissão,AC,2004
5,Energia,Emissão,AC,2005
6,Energia,Emissão,AC,2006
7,Energia,Emissão,AC,2007
8,Energia,Emissão,AC,2008
9,Energia,Emissão,AC,2009


No formato que os dados estão dispostos no dataset df_cat, nós sabemos que cada linha do dataset df_data represeta as informaões para a primeira categoria do nivel 1, no primeiro tipo de emissao, para o primeiro estado e para os anos de 2000 a 2019. A segunda linha em diante representa a segunda categoria do nivel 1, no primeiro tipo de emissao, para o primeiro estado e para os anos de 2000 a 2019, seguindo assim até o final do dataset.

Desta forma, só precisamos transpor cada linha do dataset df_data em uma única coluna, que deverá retornar um array de dimensão (5400,1), no exato formato do nosso novo dataset df_cat.

In [15]:
print('Dimensões do dataset df_data: ', df_data.shape)

Dimensões do dataset df_data:  (270, 20)


In [16]:
## Melting rows to a single column
df_data = pd.melt(df_data.T)
df_data.head()

,variable,value
0,0,8.160077e+05
1,0,8.938214e+05
2,0,1.072257e+06
3,0,1.064160e+06
4,0,8.259772e+05


In [17]:
## Passin melted column to a list
values_list = df_data.iloc[:,1].tolist()

print('Tamanho da lista de valores: ', len(values_list))

Tamanho da lista de valores:  5400


In [18]:
## Joining melted values to our final adjusted dataset
df_cat['EMISSOES'] = values_list

In [19]:
## Overview of the adjusted dataset
df_cat.head(41)

,0,1,2,ANO,EMISSOES
0,Energia,Emissão,AC,2000,8.160077e+05
1,Energia,Emissão,AC,2001,8.938214e+05
2,Energia,Emissão,AC,2002,1.072257e+06
3,Energia,Emissão,AC,2003,1.064160e+06
4,Energia,Emissão,AC,2004,8.259772e+05
5,Energia,Emissão,AC,2005,7.625486e+05
6,Energia,Emissão,AC,2006,6.468860e+05
7,Energia,Emissão,AC,2007,5.625499e+05
8,Energia,Emissão,AC,2008,5.740895e+05
9,Energia,Emissão,AC,2009,5.615789e+05


In [25]:
## Setting header names
df_cat.columns = ['setor', 'tipo', 'uf', 'ano', 'emissao_co2']

## Dropping 2019 information due to inconsistency of data
df_adjusted = df_cat.query('ano != 2019')

In [52]:
## change setor and tipo categories to upper case
regex = '\s$'
setor = df_adjusted['setor'].tolist()
tipo = df_adjusted['tipo'].tolist()
pd.options.mode.chained_assignment = None
setor =  strip_upper(setor, regex)
tipo = strip_upper(tipo, regex)

print(set(setor))
print(set(tipo))

df_adjusted.loc[:, 'setor'] = setor
df_adjusted.loc[:, 'tipo'] = tipo

{'MUDANCA DE USO DA TERRA E FLORESTA', 'RESIDUOS', 'ENERGIA', 'PROCESSOS INDUSTRIAIS', 'AGROPECUARIA'}
{'REMOCAO', 'EMISSAO'}


In [61]:
## Catching a random row to verify values
df_adjusted.iloc[1230, :]

setor          MUDANCA DE USO DA TERRA E FLORESTA
tipo                                      EMISSAO
uf                                             DF
ano                                          2014
emissao_co2                              255778.0
Name: 1294, dtype: object

In [62]:
# Returning equivalent row in original dataset
df_original.loc[(df_original['NIVEL 1'] == 'Mudança de Uso da Terra e Floresta') &
                (df_original['TERRITÓRIO'] == 'DF')]

,NIVEL 1,NIVEL 2,NIVEL 3,NIVEL 4,NIVEL 5,NIVEL 6,TIPO DE EMISSÃO,GÁS,TERRITÓRIO,IBGE,...,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,AREA2019
64,Mudança de Uso da Terra e Floresta,NaN,NaN,NaN,NaN,NaN,Emissão,CO2e (t) GWP-AR5,DF,NaN,...,220347.0,208910.0,136666.0,238594.0,255778.0,169654.0,177102.0,172721.0,146519.0,224670.42
69,Mudança de Uso da Terra e Floresta,NaN,NaN,NaN,NaN,NaN,Remoção,CO2e (t) GWP-AR5,DF,NaN,...,-372645.0,-366427.0,-391963.0,-362662.0,-375553.0,-355586.0,-350564.0,-351112.0,-350177.0,109454.82


In [63]:
## Final overview
df_adjusted.head()

,setor,tipo,uf,ano,emissao_co2
0,ENERGIA,EMISSAO,AC,2000,8.160077e+05
1,ENERGIA,EMISSAO,AC,2001,8.938214e+05
2,ENERGIA,EMISSAO,AC,2002,1.072257e+06
3,ENERGIA,EMISSAO,AC,2003,1.064160e+06
4,ENERGIA,EMISSAO,AC,2004,8.259772e+05


In [64]:
## Exporting adjusted dataset
df_adjusted.to_csv('bases-auxiliares/SEEG-ajustado.csv', index = False, header = True)